# 16 - HDFS + Spark Storage Strategies

Optymalizacja przechowywania danych na HDFS dla Spark.

**Tematy:**
- Formaty plików: Parquet vs ORC vs Avro vs CSV (benchmark)
- Kompresja: Snappy vs GZIP vs LZ4 vs ZSTD
- Partycjonowanie danych na HDFS
- Small files problem i compaction
- Block size vs partition size
- Bucketing - optymalizacja joinów

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import time

spark = SparkSession.builder \
    .appName("16_Storage_Strategies") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.1") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "7g") \
    .config("spark.driver.host", "recommender-jupyter") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .getOrCreate()

HDFS_URL = "hdfs://namenode:9000"
HDFS_BASE = f"{HDFS_URL}/data/movielens/benchmarks"
jdbc_url = "jdbc:postgresql://postgres:5432/recommender"
jdbc_props = {"user": "recommender", "password": "recommender", "driver": "org.postgresql.Driver"}

ratings = spark.read.jdbc(
    jdbc_url, "movielens.ratings", properties=jdbc_props,
    column="user_id", lowerBound=1, upperBound=300000, numPartitions=10
)
ratings.cache()
print(f"Ratings: {ratings.count()} rows")

## 2. Benchmark formatów plików

| Format | Typ | Kompresja | Column pruning | Predicate pushdown | Use case |
|--------|-----|-----------|----------------|-------------------|----------|
| **Parquet** | Kolumnowy | Snappy/GZIP/ZSTD | Tak | Tak | Analityka (domyślny Spark) |
| **ORC** | Kolumnowy | ZLIB/Snappy/LZO | Tak | Tak | Hive ecosystem |
| **Avro** | Wierszowy | Snappy/Deflate | Nie | Nie | Streaming, schema evolution |
| **CSV** | Tekst | GZIP/BZip2 | Nie | Nie | Interop, małe dane |
| **JSON** | Tekst | GZIP | Nie | Nie | API, czytelność |

In [ ]:
def get_hdfs_size(path):
    """Rozmiar pliku/katalogu na HDFS w MB."""
    try:
        fs = spark.sparkContext._jvm.org.apache.hadoop.fs.FileSystem.get(
            spark.sparkContext._jvm.java.net.URI(HDFS_URL),
            spark.sparkContext._jsc.hadoopConfiguration()
        )
        p = spark.sparkContext._jvm.org.apache.hadoop.fs.Path(path)
        return fs.getContentSummary(p).getLength() / 1024 / 1024
    except:
        return 0

# Benchmark: zapis w różnych formatach
formats = ["parquet", "orc", "csv", "json"]
write_results = []

for fmt in formats:
    path = f"{HDFS_BASE}/format_{fmt}"
    start = time.time()
    if fmt == "csv":
        ratings.write.mode("overwrite").option("header", True).csv(path)
    else:
        ratings.write.mode("overwrite").format(fmt).save(path)
    write_time = time.time() - start
    size = get_hdfs_size(path)
    write_results.append((fmt, write_time, size))
    print(f"{fmt:<10} write={write_time:.1f}s  size={size:.0f}MB")

In [ ]:
# Benchmark: odczyt + analiza w różnych formatach
print("\n=== Full scan (count) ===")
for fmt in formats:
    path = f"{HDFS_BASE}/format_{fmt}"
    start = time.time()
    if fmt == "csv":
        spark.read.option("header", True).option("inferSchema", True).csv(path).count()
    else:
        spark.read.format(fmt).load(path).count()
    t = time.time() - start
    print(f"{fmt:<10} {t:.2f}s")

print("\n=== Column pruning (select 2 columns) ===")
for fmt in formats:
    path = f"{HDFS_BASE}/format_{fmt}"
    start = time.time()
    if fmt == "csv":
        spark.read.option("header", True).option("inferSchema", True).csv(path) \
            .select("user_id", "rating").count()
    else:
        spark.read.format(fmt).load(path).select("user_id", "rating").count()
    t = time.time() - start
    print(f"{fmt:<10} {t:.2f}s")

print("\n=== Predicate pushdown (filter + aggregate) ===")
for fmt in formats:
    path = f"{HDFS_BASE}/format_{fmt}"
    start = time.time()
    if fmt == "csv":
        df = spark.read.option("header", True).option("inferSchema", True).csv(path)
    else:
        df = spark.read.format(fmt).load(path)
    df.filter(col("rating") >= 4.5).groupBy("movie_id").count().count()
    t = time.time() - start
    print(f"{fmt:<10} {t:.2f}s")

## 3. Kompresja

In [ ]:
# Benchmark kompresji dla Parquet
compressions = ["snappy", "gzip", "lz4", "zstd", "none"]

print("=== Parquet compression benchmark ===")
for comp in compressions:
    path = f"{HDFS_BASE}/compression_{comp}"
    start = time.time()
    ratings.write.mode("overwrite") \
        .option("compression", comp) \
        .parquet(path)
    write_time = time.time() - start
    size = get_hdfs_size(path)
    
    # Read benchmark
    start = time.time()
    spark.read.parquet(path).filter(col("rating") >= 4.0).count()
    read_time = time.time() - start
    
    print(f"{comp:<10} size={size:>6.0f}MB  write={write_time:.1f}s  read={read_time:.1f}s")

### Rekomendacje kompresji:
- **Snappy** (domyślna): najszybsza, umiarkowana kompresja. Hot data.
- **ZSTD**: najlepsza kompresja z rozsądną szybkością. Warm data.
- **GZIP**: dobra kompresja, wolny zapis. Cold data / archiwum.
- **LZ4**: bardzo szybka, słaba kompresja. Streaming.
- **none**: bez kompresji. Tylko gdy CPU jest bottleneckiem.

## 4. Partycjonowanie na HDFS

In [ ]:
# Partycjonowanie po roku - tworzy podkatalogi
ratings_with_year = ratings.withColumn("year", year(col("rating_timestamp")))

ratings_with_year.write.mode("overwrite") \
    .partitionBy("year") \
    .parquet(f"{HDFS_BASE}/partitioned_by_year")

# Partition pruning - czyta TYLKO potrzebny rok
print("=== Partition pruning benchmark ===")

# Bez partycjonowania (full scan)
start = time.time()
ratings_with_year.filter(col("year") == 2015).count()
no_partition = time.time() - start

# Z partycjonowaniem (partition pruning)
start = time.time()
spark.read.parquet(f"{HDFS_BASE}/partitioned_by_year") \
    .filter(col("year") == 2015).count()
with_partition = time.time() - start

print(f"Bez partycjonowania: {no_partition:.2f}s")
print(f"Z partycjonowaniem:  {with_partition:.2f}s")

## 5. Small Files Problem

Każdy plik na HDFS = ~150 bajtów metadanych w NameNode (RAM!).
1 milion małych plików = 150MB RAM NameNode.

**Problem:** `repartition(1000).partitionBy("year")` z 15 lat = 15000 plików!

In [ ]:
# Symulacja problemu małych plików
# ZŁE: dużo małych plików
spark.conf.set("spark.sql.shuffle.partitions", "200")
ratings_with_year.repartition(200) \
    .write.mode("overwrite") \
    .partitionBy("year") \
    .parquet(f"{HDFS_BASE}/small_files_bad")

# DOBRE: kontrolowana liczba plików
ratings_with_year.repartition("year") \
    .write.mode("overwrite") \
    .partitionBy("year") \
    .parquet(f"{HDFS_BASE}/small_files_good")

# Policz pliki
def count_files(path):
    fs = spark.sparkContext._jvm.org.apache.hadoop.fs.FileSystem.get(
        spark.sparkContext._jvm.java.net.URI(HDFS_URL),
        spark.sparkContext._jsc.hadoopConfiguration())
    p = spark.sparkContext._jvm.org.apache.hadoop.fs.Path(path)
    iterator = fs.listFiles(p, True)
    count = 0
    while iterator.hasNext():
        f = iterator.next()
        if not f.getPath().getName().startswith("_"):
            count += 1
    return count

bad_count = count_files(f"{HDFS_BASE}/small_files_bad")
good_count = count_files(f"{HDFS_BASE}/small_files_good")
print(f"Bez optymalizacji: {bad_count} plików")
print(f"Z repartition(year): {good_count} plików")

In [ ]:
# Compaction - łączenie małych plików w większe
# Odczytaj i zapisz ponownie z mniejszą liczbą partycji

spark.read.parquet(f"{HDFS_BASE}/small_files_bad") \
    .coalesce(10) \
    .write.mode("overwrite") \
    .parquet(f"{HDFS_BASE}/compacted")

compacted_count = count_files(f"{HDFS_BASE}/compacted")
print(f"Po compaction: {compacted_count} plików")

## 6. Bucketing - optymalizacja joinów

Bucketing pre-sortuje dane po kluczu i zapisuje do stałej liczby plików.
Przy join po tym samym kluczu - **zero shuffle!**

In [ ]:
# Zapisz ratings z bucketing po movie_id
ratings.write.mode("overwrite") \
    .bucketBy(16, "movie_id") \
    .sortBy("movie_id") \
    .saveAsTable("ratings_bucketed")

# Join bucketed tables - sprawdź plan (brak Exchange/Shuffle!)
bucketed = spark.table("ratings_bucketed")
bucketed.join(spark.table("ratings_bucketed").groupBy("movie_id").count(), "movie_id") \
    .explain()

## Zadanie końcowe

Zoptymalizuj storage dla systemu rekomendacji na HDFS:

1. Wybierz optymalny format i kompresję dla każdej warstwy:
   - Bronze: archiwum (priorytet: rozmiar)
   - Silver: częsty odczyt (priorytet: szybkość odczytu)
   - Gold: małe agregaty (priorytet: szybkość query)
2. Zaprojektuj partycjonowanie (po jakim kluczu?)
3. Zmierz: rozmiar na HDFS, czas zapisu, czas typowego query
4. Porównaj z nieoptymalną wersją (CSV, bez partycji, bez kompresji)

In [ ]:
# Twoje rozwiązanie:


In [ ]:
ratings.unpersist()
spark.stop()